In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('School Profile 2008-2024.csv')
df.head()

C:\Users\jason\AppData\Local\Temp\ipykernel_2248\1877593476.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('School Profile 2008-2024.csv')


,Calendar Year,ACARA SML ID,Location AGE ID,School AGE ID,School Name,Suburb,State,Postcode,School Sector,School Type,...,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Full Time Equivalent Enrolments,Indigenous Enrolments (%),Language Background Other Than English - Yes (%),Language Background Other Than English - No (%),Language Background Other Than English - Not Stated (%)
0,2024,40000,40003.0,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,...,25.0,16.6,410.0,193.0,217.0,409.6,6.0,8.0,92.0,0.0
1,2023,40000,40003.0,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,...,21.0,15.0,413.0,192.0,221.0,412.6,5.0,8.0,92.0,0.0
2,2022,40000,40003.0,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,...,20.0,13.7,407.0,182.0,225.0,406.6,4.0,7.0,93.0,0.0
3,2021,40000,40003.0,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,...,19.0,12.6,408.0,173.0,235.0,407.6,2.0,7.0,93.0,0.0
4,2020,40000,40003.0,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,...,21.0,13.8,398.0,168.0,230.0,397.6,2.0,6.0,94.0,0.0


In [10]:
# number of unique suburbs
df['Suburb'].nunique()

4607

In [20]:
# Build dataframe: year, school name, suburb, teaching staff per total enrolments
import numpy as np

# Helper to normalise column names for fuzzy matching
_norm = lambda s: ''.join(ch for ch in s.lower() if ch.isalnum())

def find_col(df, candidates):
    cand_norm = [_norm(c) for c in candidates]
    for col in df.columns:
        n = _norm(col)
        if n in cand_norm:
            return col
    # fallback: substring match
    for col in df.columns:
        n = _norm(col)
        if any(c in n for c in cand_norm):
            return col
    return None

# Candidate names for required columns
year_cands   = ["Calendar Year"]
name_cands   = ["School Name"]
suburb_cands = ["Suburb"]
state_cands = ["State"]
type_cands   = ["School Type"]
staff_cands  = [
    "Teaching Staff"
]
enrol_cands  = [
    "Total Enrolments"
]

# Resolve actual column names present
col_year   = find_col(df, year_cands)
col_name   = find_col(df, name_cands)
col_suburb = find_col(df, suburb_cands)
col_type   = find_col(df, type_cands)
col_staff  = find_col(df, staff_cands)
col_enrol  = find_col(df, enrol_cands)
col_state  = find_col(df, state_cands)

missing = [
    k for k, v in {
        "year": col_year,
        "school name": col_name,
        "suburb": col_suburb,
        "school type": col_type,
        "teaching staff": col_staff,
        "total enrolments": col_enrol,
        "state": col_state,
    }.items() if v is None
]
if missing:
    raise KeyError(f"Could not find required columns in dataframe: {', '.join(missing)}. Available columns: {list(df.columns)}")

# Filter rows to Secondary or Combined school types
mask_types = (
    df[col_type].astype(str).str.strip().str.lower().isin(["secondary", "combined"])  # case-insensitive
)

df_filt = df.loc[mask_types].copy()

# Ensure numeric and avoid divide-by-zero
staff_num = pd.to_numeric(df_filt[col_staff], errors='coerce')
enrol_num = pd.to_numeric(df_filt[col_enrol], errors='coerce')
ratio = staff_num / enrol_num.replace({0: np.nan})

# Assemble output
out = pd.DataFrame({
    "year": df_filt[col_year],
    "school name": df_filt[col_name],
    "suburb": df_filt[col_suburb],
    "state": df_filt[col_state],
    "teaching staff per total enrolments": ratio,
})

# # Optional: sort for readability
# try:
#     out = out.sort_values(["year", "school name"]).reset_index(drop=True)
# except Exception:
#     out = out.reset_index(drop=True)

# Display the result dataframe
print(out.isna().sum())
out = out.fillna(0)
print(out.isna().sum())

year                                     0
school name                              0
suburb                                   0
state                                    0
teaching staff per total enrolments    463
dtype: int64
year                                   0
school name                            0
suburb                                 0
state                                  0
teaching staff per total enrolments    0
dtype: int64


In [21]:
out.to_csv('school_staff_to_enrolment_ratio.csv', index=False)